In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

**Note**: This is a SBG workspace dependent code, it won't work in other manchines. Please contact Leandro Radusky if you want to install the dependencies and run this script by yourself.

Tested in python 3.8


# PERM analysis

Sample pipeline to study the stability, aggregability and other structural features of PERM protein. To be discussed and then replicated on the other biomarkers.

In [2]:
# Import the sbg workspace
import sys
sys.path.append('/home/leandro/Dropbox/workspacesbg/sbg/')

In [3]:
# Imports
from sbg.scripts.foldx.TangoHandler import TangoHandler
from sbg.structure.Structure import Structure
from sbg.structure.AlphaFoldHandler import AlphaFoldHandler
from sbg.orf.Orf import OnlineOrf
from sbg.orf.FastaHandler import FastaHandler
from sbg.pdbtools.pdb_bfactor import pdbBfactor
from sbg.common.FileHandler import FileHandler

from pyfoldx.structure import Structure as PyFoldxStructure

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

import nglview


In [4]:
# Some constants
PROTEIN_CODE = 'PERM'
PERM_UNIPROT_ACCESION = 'P05164'
MIMARK_ANTIGENS_SEQUENCES_PATH = './data/antigens.fasta'

In [5]:
# Some functions

def showProteinByBFactor(path):
    view = nglview.show_file(path)
    view.clear_representations()
    view.add_representation('cartoon', selection='protein')
    view.add_cartoon(selection="protein", color_scheme="bfactor")
    return view

## 1 - Compare the sequences of the MiMARK PERM antigen vs the UniProt entry

As a first step we will compare the sequences of the reference PERM protein versus the one we're using on the lab, to see which are the main differences. 

In [6]:
# Get the uniprot object
perm_up_obj = OnlineOrf(PERM_UNIPROT_ACCESION)

In [38]:
# Get the uniprot sequence
perm_up_seq = perm_up_obj.orf.sequence.text

In [8]:
# Get the MiMARK antigen sequence
fh = FastaHandler(MIMARK_ANTIGENS_SEQUENCES_PATH)

In [10]:
# Align the two protein sequences using biopython pairwise2
alignments = pairwise2.align.globalxx(perm_up_seq, fh.orfs['PERM'].seq)

In [11]:
# Display the best alignment nicely, indicating at the begining which protein is which
print('PERM Uniprot vs PERM MiMARK')
print("********************************************")
print()
# print the alignment in several lines, 80 positions per line
for i in range(0, len(alignments[0][0]), 80):
    print(f"UniProt -> ", alignments[0][0][i:i+80])
    print(f"MiMARK  -> ", alignments[0][1][i:i+80])
    print()

PERM Uniprot vs PERM MiMARK
********************************************

UniProt ->  MG--VP-F----F--SSL---RCM-V---------------DL-G-------------------P---------------
MiMARK  ->  M-KWV-TFISLLFLFSS-AYSR--GVFRREAHKSEIAHRFND-VGEEHFIGLVLITFSQYLQKAPYEEHAKLVKEVTDLA

UniProt ->  --CW-AG---------------G-----L-----T----A---EMKL---------L--------L---A------L--A
MiMARK  ->  KAC-VA-DESAANCDKSLHDIFGDKICALPSLRDTYGDVADCCE-K-KEPERNECFLHHKDDKPDLPPFARPEADVLCKA

UniProt ->  ---------G--L--------------L---A-----IL-----A-------TPQP-------SEG-------A-A-PAV
MiMARK  ->  FHDDEKAFFGHYLYEVARRHPYFYAPELLYYAQKYKAILTECCEAADKGACLT--PKLDGGGGS-GGGGSGGGASAAPAV

UniProt ->  LGEVDTSLVLSSMEEAKQLVDKAYKERRESIKQRLRSGSASPMELLSYFKQPVAATRTAVRAADYLHVALDLLERKLRSL
MiMARK  ->  LGEVDTSLVLSSMEEAKQLVDKAYKERRESIKQRLRSGSASPMELLSYFKQPVAATRTAVRAADYLHVALDLLERKLRSL

UniProt ->  WRRPFNVTDVLTPAQLNVLSKSSGCAYQDVGVTCPEQDKYRTITGMCNNRRSPTLGASNRAFVRWLPAEYEDGFSLPYGW
MiMARK  ->  WRRPFNVTDVLTPAQLNVLSKSSGCAYQDVGVTCPEQDKYRTITGMCNNRRSPTLGASNRAFVRWLPAEYEDG

Clearly, the longest match goes from PAVLGEVDT to PALNLASWREAS. The beginning of the two proteins are quite different, then the main region has total identity and the MiMARK antigen also presents a small poly histidine tag which is classical and does not represent any problem or suspicion.

In [39]:
# Extract the Uniprot positions of this match for further structural analysis
up_match_start = perm_up_seq.find('PAVLGEVDT')
up_match_end = perm_up_seq.find('PALNLASWREAS') + len('PALNLASWREAS')

print( f'Uniprot match start: {up_match_start} Uniprot match end: {up_match_end}')

Uniprot match start: 50 Uniprot match end: 745


## 2 - Computing protein structure features

After **manually** see the available structures for the PERM protein, I observed that all the structures represents the matched protein and that this is an enzime that functions in a monomerical fashion. When this is the scenario, we can rely on the AlphaFold model to have an "average" structure" free of PTMs, crystalizations ligands, etc, which also containg the full sequence modelled.

In [13]:
perm_pdb_structure = AlphaFoldHandler.get_model(PERM_UNIPROT_ACCESION)
perm_pdb_structure.savePDBFile(f'./data/{PERM_UNIPROT_ACCESION}_model.pdb')

### 2.1 - The interest region

In order to know which part of the model we're interested in, we will color the matching sequence of the reference and the antigen over the structure.

In [14]:
# Create a dict with zero bfactor for positions 50-745 and 1 for the other positions
mod_dict = {i: '0.000' if i >= up_match_start and i <= up_match_end else '1.000' for i in range(1, len(perm_pdb_structure.residues["A"])+1)}

In [15]:
# Map the modelled "score" to the structure
perm_mapped_mod_file_lines = pdbBfactor(perm_pdb_structure.fileLines, mod_dict)

In [16]:
# Write the mapped modelling scores to a file
FileHandler.writeLines(f'./data/{PERM_UNIPROT_ACCESION}_mapped_mod.pdb', perm_mapped_mod_file_lines)

True

In [17]:
# Show the structure with the aggregation scores
view = showProteinByBFactor(f'./data/{PERM_UNIPROT_ACCESION}_mapped_mod.pdb')
view

NGLWidget()

The region in red corresponds to the region of interest, the region in blue is different in our antigen and we will analyze it separately.

### 2.2 - the unmodelled region

In order to obtain a model for the initial part of the sequence of our antigen that doesn't match with the reference protein I modelled **manually** using FoldSeek server. 

In [18]:
# Show the structure with the aggregation scores
view = showProteinByBFactor(f'./data/begin_seq_mimark_modelled.pdb')
view

NGLWidget()

Here, in blue, we see the sites with the highest modelling confidence in blue, and the regions with less confidence turning to the yellow/red. It appears to be a small globule attached **but separated** to the protein through a small disordered region. This is maybe used somehow during the antigen production?

### 2.3 - Aggregablity propensity

For our PERM sturcture, let's study the aggregability propensity. This was performed using Tango software. Shortly, it moves through the protein in a sliding window fashion (a window of 5-7 aminoacids) computing the propensity of this peptide to aggregate as if it will be isolated (not in the context of a protein). 

In a protein, we're interested on regions with high propensity that are exposed on the surface and then will favour the binding of other aggregating agents.

In [19]:
# Compute aggregability scores
perm_agg_dict = TangoHandler.getAggregation(perm_pdb_structure,"aggregation/")

In [20]:
# Map the aggregation scores to the structure
perm_mapped_agg_file_lines = pdbBfactor(perm_pdb_structure.fileLines, perm_agg_dict["A"])

In [21]:
# Write the mapped aggregation scores to a file
FileHandler.writeLines(f'./data/{PERM_UNIPROT_ACCESION}_mapped_agg.pdb', perm_mapped_agg_file_lines)

# Show the structure with the aggregation scores
view = showProteinByBFactor(f'./data/{PERM_UNIPROT_ACCESION}_mapped_agg.pdb')
view

NGLWidget()

As we can observe, in our model, the only region **within** the interest region that has some aggregation propensity is buried in the hidrophobic core of the protein. This doesn't represent any aggregability risk. 

### 2.4 - Aggregablity propensity over the unmodelled region

Let's repeat the analysis on the region which is unique on the MiMARK antigen.

In [22]:
# Unmodelled region structure object
umr_st_obj = Structure("urm", f'./data/begin_seq_mimark_modelled.pdb')

# Compute aggregability scores
umr_agg_dict = TangoHandler.getAggregation(umr_st_obj,"aggregation/")

# Map the aggregation scores to the structure
umr_mapped_agg_file_lines = pdbBfactor(umr_st_obj.fileLines, umr_agg_dict["A"])

In [23]:
# Write the mapped aggregation scores to a file
FileHandler.writeLines(f'./data/umr_mapped_agg.pdb', umr_mapped_agg_file_lines)

# Show the structure with the aggregation scores
view = showProteinByBFactor(f'./data/umr_mapped_agg.pdb')
view

NGLWidget()

Again, there are only two regions with some aggregability propensity: the N-terminal, which does not represent major risks, and a small region which has some exposition to the surface. 

### 2.5 - Residues Stability

Now, let's study the residues stability. This analysis was performed using FoldX's AlaScan command. The obtained models were repaired using previously the RepairPDB command, which minimize the energy of the protein's side chains, without modifying the protein backbone. Running AlaScan computes the energetic changes produced by mutating each residue to Alanine (a "neutral" and "minimal" aminoacid). If, when mutated to alanine, the energy decrease, this means that the residue is "not happy" and it's provoking residue instabilities. On the other hand, when the energy increase, this means that the residue is "tying" the structure making bonds with other residues and changing them will make the proteins more unstable.

In [24]:
# This is costly, was executed once and the results saved into a file
#perm_pyfoldx_structure = PyFoldxStructure(PERM_UNIPROT_ACCESION, f'./data/{PERM_UNIPROT_ACCESION}_mapped_agg.pdb')
#perm_repair_results = perm_pyfoldx_structure.repair()
#perm_repair_results.toPdbFile(f'./data/{PERM_UNIPROT_ACCESION}_repaired.pdb')

In [25]:
perm_pyfoldx_structure = PyFoldxStructure(PERM_UNIPROT_ACCESION, f'./data/{PERM_UNIPROT_ACCESION}_repaired.pdb')

In [26]:
perm_alascan_results = perm_pyfoldx_structure.alanineScan()

Performing Alanine Scan...
cd ./.foldx_2024717163729976475/; /home/leandro/Dropbox/raduspostdoc/code/foldxEnterprise/build/Release/foldx --command=AlaScan --pdb=tmp.pdb > /dev/null 2> /dev/null
Alanine Scan finished.


In [76]:
from math import log, exp
# Some data transforming

# get a dict from the alascan results dataframe
perm_alascan_dict = perm_alascan_results.to_dict()['ddG_ala']
# Remove the res name and the underscore from the keys
perm_alascan_dict = { k[4:] : v for k,v in perm_alascan_dict.items()} 
# Convert to float all the values, and normalize to a positive scale
perm_alascan_dict = { k : float(v) for k,v in perm_alascan_dict.items()}
# Round the values to 3 decimal places
perm_alascan_dict = { k : round(v,3) for k,v in perm_alascan_dict.items()}
# Normalize to a 0-1 scale
perm_alascan_dict = { k : (v - min(perm_alascan_dict.values())) / (max(perm_alascan_dict.values()) - min(perm_alascan_dict.values())) for k,v in perm_alascan_dict.items()}
perm_alascan_dict = { int(k) : str(-log(v+0.01))[0:5] for k,v in perm_alascan_dict.items()}


In [77]:
perm_mapped_alascan_file_lines = pdbBfactor(perm_pdb_structure.fileLines, perm_alascan_dict)

In [78]:
FileHandler.writeLines(f'./data/{PERM_UNIPROT_ACCESION}_mapped_alascan.pdb', perm_mapped_alascan_file_lines)

True

In [79]:
# Show the structure with the aggregation scores
view = showProteinByBFactor(f'./data/{PERM_UNIPROT_ACCESION}_mapped_alascan.pdb')
view

NGLWidget()

Here we were expeting to find a blue buried region that could indicate some unstable region over the structure, the only one is on the sorrounding of HIS 261 and ARG 405. **These are the catalytic residues**, is normal that sorrounding residues are unhappy because this is when the reaction is happening and we cannot touch this!

(source: Catalytic Site Atlas)

### 2.6 - Residues Stability unmodelled region

Let's repeat the analysis for the MiMARK N-terminal region.

In [31]:
# This is costly, was executed once and the results saved into a file
#umr_pyfoldx_structure = PyFoldxStructure("umr", f'./data/umr_mapped_agg.pdb')
#umr_repair_results = umr_pyfoldx_structure.repair()
#umr_repair_results.toPdbFile(f'./data/umr_repaired.pdb')

In [32]:
# Perform the alascan on the repaired structure
umr_repaired_pyfoldx_structure = PyFoldxStructure("umr_repaired", f'./data/umr_repaired.pdb')
umr_repaired_alascan_results = umr_repaired_pyfoldx_structure.alanineScan()


Performing Alanine Scan...
cd ./.foldx_2024717163733433969/; /home/leandro/Dropbox/raduspostdoc/code/foldxEnterprise/build/Release/foldx --command=AlaScan --pdb=tmp.pdb > /dev/null 2> /dev/null
Alanine Scan finished.


In [80]:
# Some data transforming

# get a dict from the alascan results dataframe
umr_repaired_alascan_dict = umr_repaired_alascan_results.to_dict()['ddG_ala']
# Remove the res name and the underscore from the keys
umr_repaired_alascan_dict = { k[4:] : v for k,v in umr_repaired_alascan_dict.items()} 
# Convert to float all the values, and normalize to a positive scale
umr_repaired_alascan_dict = { k : float(v) for k,v in umr_repaired_alascan_dict.items()}
# Round the values to 3 decimal places
umr_repaired_alascan_dict = { k : round(v,3) for k,v in umr_repaired_alascan_dict.items()}

# Normalize to a 0-1 scale
umr_repaired_alascan_dict = { k : (v - min(umr_repaired_alascan_dict.values())) / (max(umr_repaired_alascan_dict.values()) - min(umr_repaired_alascan_dict.values())) for k,v in umr_repaired_alascan_dict.items()}
# Transform to a logarithmic scale
umr_repaired_alascan_dict = { k : -log(v+0.01) for k,v in umr_repaired_alascan_dict.items()}
# Round to 3 decimal places
umr_repaired_alascan_dict = { int(k) : str(round(v,3)) for k,v in umr_repaired_alascan_dict.items()}



In [81]:
umr_repaired_mapped_alascan_file_lines = pdbBfactor(umr_st_obj.fileLines, umr_repaired_alascan_dict)

In [82]:
FileHandler.writeLines(f'./data/umr_repaired_mapped_alascan.pdb', umr_repaired_mapped_alascan_file_lines)

True

In [83]:
# Show the structure with the stability scores
view = showProteinByBFactor(f'./data/umr_repaired_mapped_alascan.pdb')
view

NGLWidget()

Here, there are 3 residues that are unhappy. One is a glycine, this is the only residue that is smaller than an alanine and then to acomodate for the extra atom could generate some unstability, so it's expected. And then there are the THR 50 and the ARG 168. 

For the ARG168 there is most probably a modelling problem, it cannot happily accomodate due to a loop modelled too close, but flexible in reality.

For the THR 50, it is not happy becase the model does not contain waters and is a polar residue, favoring the alanine. 

None of them represents a problem.

# 3 - Summary and conclusions

- The sequence of the antigen is perfectly aligned with the sequence of the PERM protein in the Uniprot database in the globular domain. 
- This domain does not present high aggregation propensity.
- This domain also does not present high unstability scores.
- The unmodelled region has a high aggregation propensity on a region that is partially exposed.
- The unmodelled region has a high unstability score the same region that is partially exposed.

There is weak structural evidence that the protein will aggregate or be unstable. For the region added by Qiagen, some improvement could be made on the stability and aggregability on a partially exposed alpha helix, but I don't know if it has some importance in the antigen production and can not be touched, and even if this region aggregates, the main domain of the protein will remain "solid" and the binding with the antibody shouldn't be affected in high degree.

:(

